In [26]:
#Importamos las bibliotecas y estructuras a utilizar
import numpy as np
import pandas as pd
from typing import List, Tuple

"""
Este código comprende mi implementación del método simplex
se optó por usar Programación Orientada a Objetos con el fin
de modularizar el códico y hacer mas accesibles sus distintas funciones

"""

#Declaramos la clase Simplex
class Simplex:
    def __init__(self, c: List[float], A: List[List[float]], b: List[float], maximizar: bool = True) -> None:
        """
        Método constructor de la clase Simplex
        Se encarga de construir el objeto simplex cuando la clase es instanceada.

        Argumentos:
        c - Coeficientes de la función objetivo
        A - Matriz de restricciones, se asume que todas las desigualdades son del problema  Ax<=b
        b - Vector de recursos
        maximizar - Le dice al método si se trata de un problema de maximización o minimización

        Regresa:

        None
        """
        self.max=maximizar #Guardamos si el problema es de maximización o minimización
        if maximizar:
            self.c = np.array(c)
        else:
            #Si se trata de un problema de minimización lo tranformamos en uno de maximización
            self.c = -1 * np.array(c)
        self.A = np.array(A) #Atributo que representa la matriz de restricciones
        self.b = np.array(b) #Atributo que representa el vector de costos
        self.m: int = self.A.shape[0] #Atributo que representa el número de restricciones
        self.n: int = self.A.shape[1] #Atributo que representa el número de variables
        #Construimos las variables básicas de esta instancia del Simplex
        self.variables_basicas: List[str] = ['x' + str(i + 1) for i in range(self.n, self.n + self.m, 1)]

    def resolver_simplex(self) -> Tuple[List[float], float]:
        """
        Método Resolver Simplex
        Rutina principal para ejecutar el método Simplex

        Argumentos:
        La propia clase

        Regresa:
        (Solucion,optimo) - Solución al problema junto con el óptimo asociado a dicha solución en forma de tupla
        También muestra cada iteración del simplex en un dataframe de pandas

        """
        tabla_simplex: np.ndarray = self.constructor_de_la_tabla_simplex() #Creamos la tabla simplex en una matriz
        iteracion: int = 1 #Inicializamos un contador para llevar la cuenta de las iteraciones
        while True: 
            print("Tabla asociada a la iteración número: ", iteracion)
            self.mostrar_tabla(tabla_simplex)
            columna_pivote: int = np.argmin(tabla_simplex[-1, :-1]) #Buscamos la columna pivote de la tabla
            if np.all(tabla_simplex[-1, :-1] >= 0): #Verificamos si ya no quedan ninguna entrada negativa en el renglón de la función objetivo
                break #De no haberlo rompemos el ciclo del método
            renglon_pivote: int = self.buscar_renglon_pivote(tabla_simplex, columna_pivote) #Buscamos el renglón pivote con el método del mismo nombre
            if renglon_pivote == -1:  #Si el renglón es -1 es porque no se cumple que haya cuanto menos un cociente positivo y finito
                print("El problema es un problema de región factible no acotada. ") #Desplegamos un mensaje de que el problema no es un problema acotado
                return [], np.inf #Regresamos por conveniencia una solución vacia y un óptimo en el infinito
            self.actualizar_tabla(tabla_simplex, renglon_pivote, columna_pivote) #Actualizamos la tabla simplex
            self.actualizar_variables_basicas(renglon_pivote, columna_pivote) #Actualizamos la lista de variables básicas del objeto en esta iteración
            iteracion += 1 #Incrementamos nuestro contador asociado al número de la iteración
        print("Tabla asociada a la última iteración que es la número: ", iteracion)
        self.mostrar_tabla(tabla_simplex)#Mostramos la tabla final
        #Construimos nuestro vector solución asignando en la posición de las últimas variables básicas su valor encontrado
        #en la tabla simplex
        auxiliar: List[float] = tabla_simplex[:-1, -1].tolist()
        Solucion: List[float] = []
        for i in range(self.n+self.m):
            indices_no_cero=[int(i[1])-1 for i in self.variables_basicas]
            if (i in indices_no_cero):
               Solucion.append(auxiliar.pop(0))
            else:
               Solucion.append(0)
        #Modificamos el signo del óptimo de acuerdo a la modificación que realizamos al inicio si es que el problema fuese de minimización
        optimo: float = tabla_simplex[-1, -1] if self.max else -tabla_simplex[-1,-1]
        #Regresamos la solución del problema junto con el óptimo
        return Solucion, optimo

    def constructor_de_la_tabla_simplex(self) -> np.ndarray:
        """
        Método constructor de la tabla simplex
        Este método se encarga de construir la tabla simplex en forma de un arreglo de numpy

        Argumentos:
        La propia clase

        Regresa

        tabla - Arreglo de numpy que representa la tabla simplex del problema dado

        """
        #Inicializamos un arreglo en ceros el cual tenga como simensión la cantidad de restricciones por la cantidad de variables 
        #incluyendo las de holgura al pasar el problema a forma estándar
        tabla: np.ndarray = np.zeros((self.m + 1, self.n + self.m + 1))
        #Agregamos la parte de la tabla asociada a la matriz de restricciones
        tabla[:self.m, :self.n] = self.A
        #Agregamos la parte asociada a las holguras que es la parte que es una submatriz identidad
        tabla[:self.m, self.n:self.n + self.m] = np.eye(self.m)
        #Agregamos el vector de recursos a la tabla
        tabla[:self.m, -1] = self.b
        #Finalmente agregamos el vector de costos en el último renglón ya despejado como en clase, decidí ponerlo hasta abajo puesto
        #que me es mas cómodo de leer al final cuando acaba el algoritmo
        tabla[-1, :self.n] = -self.c
        #Regresamos la tabla construida
        return tabla

    def buscar_renglon_pivote(self, tabla_simplex: np.ndarray, columna_pivote: int) -> int:
        """
        Método Buscar Renglón pivote

        Se encarga de aplicar la regla del cociente y elegir el renglón pivote.
        
        Argumentos:
        tabla - tabla simplex asosciada al problema en forma de arreglo de numpy
        columna_pivote - entero que representa el número de la columna pivote calculado antes, la requiere para aplicar la regla del cociente

        Regresa:
        renglón pivote - Entero que representa el renglón pivote, en caso de ser -1 implica que no hubo ningún cociente no negativo finito y no hay pivote
        """
        
        #En primera instancia calculamos los cocientes
        cocientes: np.ndarray = tabla_simplex[:-1, -1] / tabla_simplex[:-1, columna_pivote]
        #Guardamos en un arreglo aquellas posiciones en las que los cocientes sean positivos 
        cocientes_positivos: np.ndarray = cocientes > 0
        #Hacemos lo mismo pero ahora con una intersección entre aquellos que sean positivos y no infinito (es infinito cuando divide por 0)
        cocientes_positivos_no_inf: np.ndarray = np.isfinite(cocientes) & cocientes_positivos
        #Si existe algun valor en la intersección anterior que cumpla las condiciones entonces procedemos a encontrar cual de ellos es el pivote, si no regresamos -1 
        if np.any(cocientes_positivos_no_inf):
            #Buscamos los indices donde se cumplen ambas condiciones que planteamos antes (finito y no negativo) con np.where
            indices_cocientes_positivos = np.where(cocientes_positivos_no_inf)[0]
            #Inicializamos la variable renglón pivote en el primer indice
            renglon_pivote: int = indices_cocientes_positivos[0]
            #En este ciclo comparamos el resto de coeficientes:
            for indice in indices_cocientes_positivos: #Iteramos sobre los indices encontrados que cumplen la condición
                if cocientes[indice] < cocientes[renglon_pivote]: #Si el cociente en ese indice es menor cambiamos el renglón pivote
                    renglon_pivote = indice
                #He decidido implementar de igual manera la regla de Bland para anticiclado a recomendación de la bibliografía para los casos en los que se da un empate
                # es muy sencilla puesto que de haber un empate toma el indice del cociente mas de la variable básica del numeral mas pequeño.
                elif cocientes[indice] == cocientes[renglon_pivote] and self.variables_basicas[indice] > self.variables_basicas[renglon_pivote]:
                    renglon_pivote = indice
            #Regresamos el pivote
            return renglon_pivote
        #Si no hay pivote regresamos -1
        else:
            return -1

    def actualizar_tabla(self, tabla_simplex: np.ndarray, renglon_pivote: int, columna_pivote: int) -> None:
        """
        Método actualizar tabla

        Se encarga de realizar las operaciones básicas de Gauss Jordan sobre los demás renglones

        Argumentos:
        tabla - Arreglo de numpy que representa la tabla asociada al problema
        renglón pivote - Entero que representa el renglón pivote en esta iteración
        columna pivote - Entero que representa la columna pivote en esta iteración

        Regresa:
        None

        """
        tabla_simplex[renglon_pivote, :] /= tabla_simplex[renglon_pivote, columna_pivote] #Obtenemos el 1 en el pivote
        for renglon in range(self.m + 1): #Iteramos ahora sobre los renglones de la tabla
            #En todos los renglones distintos al renglon pivote hacemos 0 en lo que este en la columna pivote (Gauss Jordan)
            tabla_simplex[renglon, :] = tabla_simplex[renglon, :] - (tabla_simplex[renglon, columna_pivote] * tabla_simplex[renglon_pivote, :]) if renglon != renglon_pivote else tabla_simplex[renglon, :]

    def actualizar_variables_basicas(self, renglon_pivote: int, columna_pivote: int) -> None:
        """
        Método actualizar variables básicas

        Se encarga de actualizar la lista atributo de variables básicas de la clase según el renglón y la columna pivote

        Argumentos:
        renglón pivote - entero que denota el renglón pivote
        columna pivote - entero que denota la columna pivote

        Regresa:
        None
        
        """
        #Simplemente construimos un string que represente la nueva variable en esta categoria para la etiqueta de la tabla
        self.variables_basicas[renglon_pivote] = 'x' + str(columna_pivote + 1)

    def mostrar_tabla(self, tabla_simplex: np.ndarray) -> None:
        """
        Método mostrar tabla

        Muestra la tabla simplex en un data frame de pandas.

        Argumentos:
        tabla simplex - La tabla en esta iteración asociada al problema a solucionar

        Regresa:
        None
        
        """
        df = pd.DataFrame(tabla_simplex)
        df.columns = [f"x{i+1}" for i in range(self.n)] + self.variables_basicas + ["b"]
        df.index = self.variables_basicas + ["z"]
        display(df)
    


A continuación se muestran una serie de problemas con los que decidí probar mi programa, algunos los realizamos en clase.

In [27]:
# Ejemplo solución degenerada
c: List[float] = [3, 9]
A: List[List[float]] = [[1, 4],
                         [1, 2]]
b: List[float] = [8, 4]
instancia_simplex_ejemplo_1: Simplex = Simplex(c, A, b, maximizar=True)
solucion, optimo = instancia_simplex_ejemplo_1.resolver_simplex()
print("El vector solución es:", solucion)
print("El óptimo es:", optimo)

Tabla asociada a la iteración número:  1


,x1,x2,x3,x4,b
x3,1.0,4.0,1.0,0.0,8.0
x4,1.0,2.0,0.0,1.0,4.0
z,-3.0,-9.0,0.0,0.0,0.0


Tabla asociada a la iteración número:  2


,x1,x2,x3,x2,b
x3,-1.0,0.0,1.0,-2.0,0.0
x2,0.5,1.0,0.0,0.5,2.0
z,1.5,0.0,0.0,4.5,18.0


Tabla asociada a la última iteración que es la número:  2


,x1,x2,x3,x2,b
x3,-1.0,0.0,1.0,-2.0,0.0
x2,0.5,1.0,0.0,0.5,2.0
z,1.5,0.0,0.0,4.5,18.0


El vector solución es: [0, 0.0, 2.0, 0]
El óptimo es: 18.0


In [31]:
c = [ 5, 4]
A = [[6, 4],
     [1, 2],
     [-1,1],
     [0,1]]
b = [24, 6, 1,2]
#Ejercicio notas 18 de mayo

instancia_simplex_ejemplo_2: Simplex = Simplex(c, A, b, maximizar=True)
solucion, optimo = instancia_simplex_ejemplo_2.resolver_simplex()
print("El vector solución es:", solucion)
print("El óptimo es:", optimo)

Tabla asociada a la iteración número:  1


,x1,x2,x3,x4,x5,x6,b
x3,6.0,4.0,1.0,0.0,0.0,0.0,24.0
x4,1.0,2.0,0.0,1.0,0.0,0.0,6.0
x5,-1.0,1.0,0.0,0.0,1.0,0.0,1.0
x6,0.0,1.0,0.0,0.0,0.0,1.0,2.0
z,-5.0,-4.0,0.0,0.0,0.0,0.0,0.0


Tabla asociada a la iteración número:  2


/tmp/ipykernel_50878/3663350192.py:131: RuntimeWarning: divide by zero encountered in divide
  cocientes: np.ndarray = tabla_simplex[:-1, -1] / tabla_simplex[:-1, columna_pivote]


,x1,x2,x1,x4,x5,x6,b
x1,1.0,0.666667,0.166667,0.0,0.0,0.0,4.0
x4,0.0,1.333333,-0.166667,1.0,0.0,0.0,2.0
x5,0.0,1.666667,0.166667,0.0,1.0,0.0,5.0
x6,0.0,1.000000,0.000000,0.0,0.0,1.0,2.0
z,0.0,-0.666667,0.833333,0.0,0.0,0.0,20.0


Tabla asociada a la iteración número:  3


,x1,x2,x1,x2,x5,x6,b
x1,1.0,0.0,0.250,-0.50,0.0,0.0,3.0
x2,0.0,1.0,-0.125,0.75,0.0,0.0,1.5
x5,0.0,0.0,0.375,-1.25,1.0,0.0,2.5
x6,0.0,0.0,0.125,-0.75,0.0,1.0,0.5
z,0.0,0.0,0.750,0.50,0.0,0.0,21.0


Tabla asociada a la última iteración que es la número:  3


,x1,x2,x1,x2,x5,x6,b
x1,1.0,0.0,0.250,-0.50,0.0,0.0,3.0
x2,0.0,1.0,-0.125,0.75,0.0,0.0,1.5
x5,0.0,0.0,0.375,-1.25,1.0,0.0,2.5
x6,0.0,0.0,0.125,-0.75,0.0,1.0,0.5
z,0.0,0.0,0.750,0.50,0.0,0.0,21.0


El vector solución es: [3.0, 1.4999999999999998, 0, 0, 2.5000000000000004, 0.5000000000000002]
El óptimo es: 21.0


In [32]:
c = [5, -4, 6, -8]
A = [[1, 2, 2, 4],
     [2, -1, 1, 2],
     [4, -2, 1, -1]]
b = [40, 8, 10]
#Ejercicio notas 25 de mayo

instancia_simplex_ejemplo_3: Simplex = Simplex(c, A, b, maximizar=False)
solucion, optimo = instancia_simplex_ejemplo_3.resolver_simplex()
print("El vector solución es:", solucion)
print("El óptimo es:", optimo)

Tabla asociada a la iteración número:  1


,x1,x2,x3,x4,x5,x6,x7,b
x5,1.0,2.0,2.0,4.0,1.0,0.0,0.0,40.0
x6,2.0,-1.0,1.0,2.0,0.0,1.0,0.0,8.0
x7,4.0,-2.0,1.0,-1.0,0.0,0.0,1.0,10.0
z,5.0,-4.0,6.0,-8.0,0.0,0.0,0.0,0.0


Tabla asociada a la iteración número:  2


,x1,x2,x3,x4,x5,x4,x7,b
x5,-3.0,4.0,0.0,0.0,1.0,-2.0,0.0,24.0
x4,1.0,-0.5,0.5,1.0,0.0,0.5,0.0,4.0
x7,5.0,-2.5,1.5,0.0,0.0,0.5,1.0,14.0
z,13.0,-8.0,10.0,0.0,0.0,4.0,0.0,32.0


Tabla asociada a la iteración número:  3


,x1,x2,x3,x4,x2,x4,x7,b
x2,-0.750,1.0,0.0,0.0,0.250,-0.50,0.0,6.0
x4,0.625,0.0,0.5,1.0,0.125,0.25,0.0,7.0
x7,3.125,0.0,1.5,0.0,0.625,-0.75,1.0,29.0
z,7.000,0.0,10.0,0.0,2.000,0.00,0.0,80.0


Tabla asociada a la última iteración que es la número:  3


,x1,x2,x3,x4,x2,x4,x7,b
x2,-0.750,1.0,0.0,0.0,0.250,-0.50,0.0,6.0
x4,0.625,0.0,0.5,1.0,0.125,0.25,0.0,7.0
x7,3.125,0.0,1.5,0.0,0.625,-0.75,1.0,29.0
z,7.000,0.0,10.0,0.0,2.000,0.00,0.0,80.0


El vector solución es: [0, 6.0, 0, 7.0, 0, 0, 29.0]
El óptimo es: -80.0


In [33]:
#ejemplo youtube
c=[50,80]
A=[[1,2],
   [1,1]]
b=[120,90]

instancia_simplex_ejemplo_4: Simplex = Simplex(c, A, b, maximizar=True)
solucion, optimo = instancia_simplex_ejemplo_4.resolver_simplex()
print("El vector solución es:", solucion)
print("El óptimo es:", optimo)

Tabla asociada a la iteración número:  1


,x1,x2,x3,x4,b
x3,1.0,2.0,1.0,0.0,120.0
x4,1.0,1.0,0.0,1.0,90.0
z,-50.0,-80.0,0.0,0.0,0.0


Tabla asociada a la iteración número:  2


,x1,x2,x2,x4,b
x2,0.5,1.0,0.5,0.0,60.0
x4,0.5,0.0,-0.5,1.0,30.0
z,-10.0,0.0,40.0,0.0,4800.0


Tabla asociada a la iteración número:  3


,x1,x2,x2,x1,b
x2,0.0,1.0,1.0,-1.0,30.0
x1,1.0,0.0,-1.0,2.0,60.0
z,0.0,0.0,30.0,20.0,5400.0


Tabla asociada a la última iteración que es la número:  3


,x1,x2,x2,x1,b
x2,0.0,1.0,1.0,-1.0,30.0
x1,1.0,0.0,-1.0,2.0,60.0
z,0.0,0.0,30.0,20.0,5400.0


El vector solución es: [30.0, 60.0, 0, 0]
El óptimo es: 5400.0
